In [ ]:
!pip install darts

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler,MissingValuesFiller
from darts.models import NLinearModel, ExponentialSmoothing,TCNModel, RNNModel,TFTModel,NBEATSModel,NaiveDrift,NaiveSeasonal,NHiTSModel
from darts.metrics import mape, r2_score,mae
from darts.utils.statistics import check_seasonality, plot_acf
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import QuantileRegression
from pytorch_lightning.callbacks import Callback, EarlyStopping
# import optuna
# from optuna.integration import PyTorchLightningPruningCallback
# from optuna.visualization import (
#     plot_optimization_history,
#     plot_contour,
#     plot_param_importances,
# )
import torch
import random
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

In [ ]:
df_name = "/kaggle/input/hackathon-online-cryptocurrency-prediction/train.csv"

In [ ]:
import pandas as pd
from darts.utils.missing_values import fill_missing_values
# Read a pandas DataFrame
df = pd.read_csv(df_name, delimiter=",")[500:]
df["price"].plot()

In [ ]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_series(df["price"],fill_missing_dates=True)
# series_ma = series.add_datetime_attribute("week")
#series = series.stack(series_ma)
# Set aside the last 36 months as a validation series
val_len = 500

train = series[: -(2 * val_len)]
val = series[-(2 * val_len) : -val_len]
test = series[-val_len:]

In [ ]:
train.plot()

In [ ]:
model = ExponentialSmoothing()

In [ ]:
backtest_series_ets = model.historical_forecasts(
        series=series,
        start=0.5,
        forecast_horizon=16,
        stride=16,
        retrain=True,
        verbose=True,
)

In [ ]:
import matplotlib.pyplot as plt

series.plot()
backtest_series_ets.plot(label="forecast", low_quantile=0.05, high_quantile=0.9)
plt.title("MAE: {:.2f}%".format(mae(backtest_series_ets, val)))
plt.legend()

In [ ]:
def eval_model(model, n, series, val_series):
    pred_series = model.predict(n=n)
    plt.figure(figsize=(8, 5))
    series.plot(label="actual")
    pred_series.plot(label="forecast")
    plt.title("MAE: {:.2f}%".format(mae(pred_series, val_series)))
    plt.legend()

In [ ]:
model = NLinearModel(
    input_chunk_length=100,
    output_chunk_length=5,
    n_epochs=200,
    batch_size=4000,
    force_reset=True,
    save_checkpoints=True,
    model_name="nlinear_model",
)

In [ ]:
model.fit(
        series=train,
        val_series=val,
)

In [ ]:
model = NLinearModel.load_from_checkpoint("nlinear_model")
backtest_series = model.historical_forecasts(
        series=series,
        start=0.5,
        forecast_horizon=16,
        stride=16,
        retrain=False,
        verbose=True,
)

In [ ]:
import matplotlib.pyplot as plt

series.plot()
backtest_series.plot(label="forecast", low_quantile=0.05, high_quantile=0.9)
plt.title("MAE: {:.2f}%".format(mae(backtest_series, val)))
plt.legend()

In [ ]:
eval_model(model, 1500, series, val)

In [ ]:
series.plot(label="actual")
model.predict(953,series).plot()
plt.legend()

In [ ]:
pred = list(model.predict(953,series).pd_dataframe()["price"])

In [ ]:
pred[:10]

In [ ]:
df_pred = pd.read_csv("/kaggle/input/hackathon-online-cryptocurrency-prediction/sample_submission.csv")

In [ ]:
df_pred["price"][5:] = pred[5:]

In [ ]:
df_pred.to_csv("test.csv",index = 0)